In [1]:
import numpy as np
import pandas as pd
# Use another model to get new features

In [2]:
data = pd.read_csv("crop_production.csv")
df = pd.DataFrame(data)
df = df.dropna()
cols = [-1,-2]
y = df[df.columns[cols]]
df = df.drop(['Production',"District_Name","Crop_Year"],axis = 1)
y["Result"] = y["Production"]/y['Area']
y = y.drop(['Area','Production'],axis = 1)

In [3]:
df

,District_Name,Season,Crop,Area
0,NICOBARS,Kharif,Arecanut,1254.0
1,NICOBARS,Kharif,Other Kharif pulses,2.0
2,NICOBARS,Kharif,Rice,102.0
3,NICOBARS,Whole Year,Banana,176.0
4,NICOBARS,Whole Year,Cashewnut,720.0
...,...,...,...,...
246086,PURULIA,Summer,Rice,306.0
246087,PURULIA,Summer,Sesamum,627.0
246088,PURULIA,Whole Year,Sugarcane,324.0
246089,PURULIA,Winter,Rice,279151.0


In [4]:
y

,Result
0,1.594896
1,0.500000
2,3.147059
3,3.642045
4,0.229167
...,...
246086,2.617647
246087,0.738437
246088,50.154321
246089,2.141848


In [5]:
df = pd.get_dummies(df, columns = ['State_Name', 'Season','Crop'])
df
# df[0]["District_Name_NICOBARS"]

,Area,District_Name_24 PARAGANAS NORTH,District_Name_24 PARAGANAS SOUTH,District_Name_ADILABAD,District_Name_AGAR MALWA,District_Name_AGRA,District_Name_AHMADABAD,District_Name_AHMEDNAGAR,District_Name_AIZAWL,District_Name_AJMER,...,Crop_Turmeric,Crop_Turnip,Crop_Urad,Crop_Varagu,Crop_Water Melon,Crop_Wheat,Crop_Yam,Crop_other fibres,Crop_other misc. pulses,Crop_other oilseeds
0,1254.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,102.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,176.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,720.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246086,306.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
246087,627.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
246088,324.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
246089,279151.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size = .15, random_state = 18)
X_train , X_cv , y_train , y_cv = train_test_split(df, y, test_size = .10, random_state = 18)

In [7]:
X_train

,Area,District_Name_24 PARAGANAS NORTH,District_Name_24 PARAGANAS SOUTH,District_Name_ADILABAD,District_Name_AGAR MALWA,District_Name_AGRA,District_Name_AHMADABAD,District_Name_AHMEDNAGAR,District_Name_AIZAWL,District_Name_AJMER,...,Crop_Turmeric,Crop_Turnip,Crop_Urad,Crop_Varagu,Crop_Water Melon,Crop_Wheat,Crop_Yam,Crop_other fibres,Crop_other misc. pulses,Crop_other oilseeds
25482,180.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
46669,99.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5825,289.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
218689,8861.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
179540,2367.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,242.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
35737,1469.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
93160,6687.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
96393,13376.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

param_grid_gb = {
    "n_estimators": [10, 100, 200, 500, 1000, 1200],
    "max_depth": [None, 5, 10, 20, 30],
    "max_features": ["auto", "sqrt"],
    "min_samples_split": [2,4,6],
    "min_samples_leaf": [1, 2, 4]}


model = XGBRegressor()
mse_grid = GridSearchCV(estimator=model, param_grid=param_grid_gb, scoring='neg_mean_squared_error', cv=5, verbose=1)

mse_grid.fit(X_train, y_train, eval_set=[(X_cv, y_cv)], early_stopping_rounds=20,verbose = False)

print("Best parameters:", mse_grid.best_params_)


# Procedure: First tune min_child_weight and max_depth
# Tune gamma
# Tune subsample and colsample
# Tune regularization (alpha paramater)
# Tune the estimators as required

# Previous run (27m 35.4s) Best parameter:  {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1}
# Best parameter:  {'colsample_bytree': 1e-05, 'gamma': 0, 'learning_rate': 1e-07, 'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 100, 'reg_alpha': 1e-07, 'subsample': 1}


Fitting 5 folds for each of 20 candidates, totalling 100 fits


KeyboardInterrupt: 

In [9]:
# Best parameter:  {'colsample_bytree': 1, 'gamma': 1e-05, 'learning_rate': 1e-07, 'max_depth': 4, 'min_child_weight': 12, 'n_estimators': 100, 'reg_alpha': 1e-08, 'subsample': 0.9}
from xgboost import XGBRegressor

model = XGBRegressor(n_estimators=1000,eta = 1e-03)
# eval_set = [(X_test,y_test)]
model.fit   (X_train,y_train,eval_set=[(X_train,y_train),(X_test,y_test)],early_stopping_rounds = 10)

# print("Best parameter: ", mse_grid.best_params_)

KeyboardInterrupt: 

In [10]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train,y_train)
# preds = model.predict(X_test)

c:\Python311\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [93]:
# training_error = 0
from math import sqrt
from sklearn.metrics import mean_squared_error
print("Training error:")
print(sqrt(mean_squared_error(y_train,model.predict(X_train))))
print("Test error:")
print(sqrt(mean_squared_error(y_test,model.predict(X_test))))
# Previously
# Training error:
# 2041076666.4211206
# Test error:
# 71631908865818.11
# Overfit
# Now
# Training error:
# 410632.44332906447
# Test error:
# 681743.0108548584
# Ok

Training error:
635.7582325304996
Test error:
861.047758404154


In [ ]:
from sklearn.metrics import r2_score
preds = model.predict(X_test)
r = r2_score(y_test,preds)
print("R2score when we predict using XGB Regressor is ",r)

R2score when we predict using XGB Regressor is  0.16669160168206032


In [ ]:
# This will be used for user inputs
# State_Name = states['Andaman and Nicobar Islands']
s = 'Kharif'
District_Name = districts['NICOBARS']
# Crop_Year = 2000
Season = season_to_number[s]
Crop = crops['Arecanut']
Area = 1254.00
Len = len_of_months[s]
# print(State_Name,District_Name,Crop_Year,Season,Crop,Area)
X_predict = np.zeros((1,3))
X_predict[0] = np.array([District_Name,Season,Crop])
X_pred_normalized = scaler.transform(X_predict)

Production = model.predict(X_pred_normalized)
print(X_pred_normalized)
print(Production*Area)

[[-1.65923103 -0.97610996 -1.30967773]]
[1451.1882]


In [ ]:
import pickle
filename = "finalized_model.sav"
# pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [ ]:
import pickle
filename = "finalized_model2.sav"
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
#  Does not perform well on small aread like 1 10 100
# load the model from disk
# model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [ ]:
import pickle
filename = "finalized_model3.sav"
# pickle.dump(model, open(filename, 'wb'))
# model = XGBRegressor(n_estimators=1000,eta = 1e-03)

# some time later...
#  Does not perform well on small aread like 1 10 100
# load the model from disk
model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [ ]:
import pickle
filename = "finalized_model4.sav"
# pickle.dump(model, open(filename, 'wb'))
 
# some time later...
#  Does not perform well on small area like 1 10 100
# load the model from disk
model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [ ]:
import pickle
filename = "finalized_model5.sav"
pickle.dump(model, open(filename, 'wb'))

# model = pickle.load(open(filename, 'rb'))